# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [13]:
class PolynomialQuotientRing:
    def __init__(self, mod):
        self.mod = mod  # 17
        self.degree = 3  # x^4 + 1 → degree ≤ 3

    def reduce_poly(self, poly):
        """Reduce polynomial modulo X^4 + 1 and mod 17."""
        reduced = poly[:4] if len(poly) >= 4 else poly + [0] * (4 - len(poly))
        reduced = reduced[:4]  # Truncate to degree 3
        
        # Reduce higher-degree terms using X^4 ≡ -1
        for i in range(4, len(poly)):
            coeff = poly[i]
            power = i // 4
            rem = i % 4
            reduced[rem] = (reduced[rem] + (-1)**power * coeff) % self.mod
        
        return [c % self.mod for c in reduced]

    def add(self, p1, p2):
        """Add two polynomials."""
        max_len = max(len(p1), len(p2))
        p1 = p1 + [0] * (max_len - len(p1))
        p2 = p2 + [0] * (max_len - len(p2))
        result = [(a + b) % self.mod for a, b in zip(p1, p2)]
        return self.reduce_poly(result)

    def multiply(self, p1, p2):
        """Multiply two polynomials with reduction."""
        result = [0] * (len(p1) + len(p2) - 1)
        for i, a in enumerate(p1):
            for j, b in enumerate(p2):
                result[i + j] = (result[i + j] + a * b) % self.mod
        return self.reduce_poly(result)

In [14]:
import secrets as sc

ring = PolynomialQuotientRing(17)

def generate_uniform_poly():
    """For matrix A: coefficients in 0-16."""
    return [sc.randbelow(17) for _ in range(4)]

def generate_small_poly():
    """For s, e, r: coefficients in {-1, 0, 1} (centered binomial)."""
    coefficients = []
    for _ in range(4):
        a, b = sc.choice([0, 1]), sc.choice([0, 1])
        coefficients.append(a - b)
    return ring.reduce_poly(coefficients)

## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [15]:
def key_gen():
    """Generate public/private keys."""
    # Uniform matrix A (coefficients 0-16)
    A = [[generate_uniform_poly() for _ in range(2)] for _ in range(2)]
    
    # Secret s and error e (small coefficients)
    s = [generate_small_poly() for _ in range(2)]
    e = [generate_small_poly() for _ in range(2)]
    
    # Compute t = A*s + e
    t = []
    for i in range(2):
        row_sum = [0]
        for j in range(2):
            product = ring.multiply(A[i][j], s[j])
            row_sum = ring.add(row_sum, product)
        t_i = ring.add(row_sum, e[i])
        t.append(t_i)
    
    return A, t, s

### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [16]:
def encrypt(A, t, m):
    """Encrypt message m (list of 4 bits)."""
    # Small coefficients for r, e1, e2
    r = [generate_small_poly() for _ in range(2)]
    e1 = [generate_small_poly() for _ in range(2)]
    e2 = generate_small_poly()

    # Transpose A
    A_T = [[A[j][i] for j in range(2)] for i in range(2)]

    # Compute u = A^T * r + e1
    u = []
    for i in range(2):
        row_sum = [0]
        for j in range(2):
            product = ring.multiply(A_T[i][j], r[j])
            row_sum = ring.add(row_sum, product)
        u_i = ring.add(row_sum, e1[i])
        u.append(u_i)

    # Compute v = t^T * r + e2 + 8*m
    v = e2.copy()
    for i in range(2):
        product = ring.multiply(t[i], r[i])
        v = ring.add(v, product)
    m_scaled = [coeff * 8 for coeff in m]
    v = ring.add(v, m_scaled)
    
    return (u, v)

### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [17]:
def decrypt(c, s):
    """Decrypt ciphertext c using secret key s."""
    u, v = c
    
    # Compute s^T * u
    sTu = [0]
    for i in range(2):
        product = ring.multiply(u[i], s[i])
        sTu = ring.add(sTu, product)
    
    # Compute v - s^T * u
    decrypted = ring.add(v, ring.multiply(sTu, [-1]))
    
    # Thresholding: decode 1 if coefficient ∈ [5, 12], else 0
    m_decoded = []
    for coeff in decrypted:
        c_mod = coeff % 17
        if 5 <= c_mod <= 12:  # Wider tolerance around 8.5 (9 ± 4)
            m_decoded.append(1)
        else:
            m_decoded.append(0)
    return m_decoded

### Testy

In [18]:
import secrets as sc

success = 0
for i in range(1000):
    output = []
    A,t,s = key_gen()
    
    m=[sc.choice((0,1)) for k in range(4)]
    
    c = encrypt(A,t,m)
    m_n = decrypt(c,s)

    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')


Success rate: 81.0 %
